In [3]:
from bs4 import BeautifulSoup
import requests

book_titles = []
book_descriptions = []
book_genres = []

page = requests.get("https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s?page=1")
soup = BeautifulSoup(page.content, 'html.parser')

<!DOCTYPE html>
<html class="desktop withSiteHeaderTopFullImage">
 <head>
  <title>
   Best Books of the Decade: 2010s (7271 books)
  </title>
  <meta content="7,271 books based on 13426 votes: The Fault in Our Stars by John Green, Divergent by Veronica Roth, Mockingjay by Suzanne Collins, Gone Girl by Gillian F..." name="description"/>
  <meta content="telephone=no" name="format-detection"/>
  <link href="https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s" rel="canonical"/>
  <script type="text/javascript">
   var ue_t0=window.ue_t0||+new Date();
  </script>
  <script type="text/javascript">
   var ue_mid = "A1PQBFHBHS6YH1";
    var ue_sn = "www.goodreads.com";
    var ue_furl = "fls-na.amazon.com";
    var ue_sid = "794-9358002-1865998";
    var ue_id = "4AMN6C84DJSXPW690WPR";

    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.p